##MIE 1512 
Identifying user roles based on user behaviors of Github
======
Xu Zhang

1003177732

Introduction
==
Github is now the largest software developing platform. As a version control and source code management system, it also assembles many other features like watch, issue comment and commit comment etc. These features enable programmers to better collaborate and improve the communication. As there are various types of behaviors for every Github user, it would be interesting to see if the users can be clustered into different groups while each group has some special traits that distinguish from others. To achieve this, I proposed that there might be some different user types based on the user behavior. As users on Github have quite different backgrounds, level of skills and behavior preferences, users could have roles like contributors(make major contributions to the repositories), issue fixers(find and fix issues in the project), inactive users(not frequently use Github), watchers(mainly user github as a source of code and do not make any contributions).

With this motivation of identify user roles and its composition of Github, the paper of "Identifying user behavior in online social networks"[1] is chosen as the main reference of my research. In this paper, the authors chose youtube users and construct features of the users using both individual attributes and social features. They used these features and cluster youtube users into 5 main groups and define the role of each group. They also chose K-means as the clustering algorithm and Beta-CV to determine the number of K. The methods of this paper would be adopted in my own research to cluster users of Github.

Unlike the literature I chose, Github users have so many different individual attributes that could be scrached from the user behaviors. For example, in the literature, there are only 5 individual attributes: Uploads, Watches, Channel views, Join date and Age. However, on Github, there are so many more types of behavior rather than just uploads and views. As a collaborative software developing platform, Github has many important features like fork, push, pull request, issue comment etc. Therefore, instead of choosing both social attributes and individual attributes, I chose to use the number of different type of behavior a user performed in a given time period to define user features. 

Starting from above assumptions and methods, in this research, I chose to use the data from githubArchive, which keeps the record of every event happend on Github since 2/12/2011. The data of Jan 2017 is used in this project and the number of each type of behavior performmed by each user are extracted from these raw data. Details of the data pre-pocessing can be found in the file DataExtraction.html

The main goals of this research are:

1. Identify different users based on the user behavior.

2. Explore features of each group's users, discover the behavior pattern of each cluster of users.

3. Calculate the composition of users'groups on Github based on clustering results. Analyze the roles of these groups and their composition.

1. Processing and Explore the data
====

##1.1 Load the data. 

The raw data is all events happened in January 2017 with total size about 80GB, but the data loaded here is the already transfromed data that records the number of events performed by each user in the first month.

There are 973887 users recorded in the Janaray 2017 with in total 14 kinds of activities on github.

In [5]:
import urllib
urllib.urlretrieve("https://github.com/ZXdatascience/MIE-1512/raw/master/output_1.csv", "/tmp/output1.csv")
urllib.urlretrieve("https://github.com/ZXdatascience/MIE-1512/raw/master/output_2.csv", "/tmp/output2.csv")
## download the data I uploaded to Github

In [6]:
JanDF1 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('file:/tmp/output1.csv')
JanDF2 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('file:/tmp/output2.csv')

In [7]:
JanDF=JanDF1.unionAll(JanDF2)
JanDF.describe().show()

As is shown from above summary, it can be seen that many users have events types that occured over 1000 times in January. It is not reasonable as there is no need for a user to perform a event over 1000 times in a month. Thus, these users should be wiped out as outliers.

In [9]:
Jan_DF= JanDF.filter("PushEvent<1000").filter("PullRequestEvent<1000").filter("IssueCommentEvent<1000").filter("PullRequestReviewCommentEvent<1000").filter("ReleaseEvent<1000").filter("IssuesEvent<1000").filter("WatchEvent<1000").filter("DeleteEvent<1000").filter("CreateEvent<1000")
## filter the outliers

In [10]:
Jan_DF.describe().show()

In [11]:
display(Jan_DF)

Above is the sample of the data I will use for clustering. The first column is the actorID, representing the corresponding user. The other 14 collumns are features a user can have. In other words, these are the number of different event types that performed by each user in January. 

Also from the data presented, events like PushEvent, CreateEvent and IssueCommentEvent have higher average values than others. This indicate that these events are more popular events that users performed more frequently.

##1.2 Normalize the data

Because the features of user have different level of magnitude, to make the result of Kmeans more convincing, I choose to normalize the data by scaling all the data to [0,1] according to the maximum number and minimum number in each column.

In [14]:
from pyspark.ml.linalg import Vectors
Jan_vector = Jan_DF.rdd.map(lambda data: [data[0], Vectors.dense([float(c) for c in data[1:]])])

In [15]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors
Jan_dataframe = spark.createDataFrame(Jan_vector, ["id", "features"])
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

scalerModel = scaler.fit(Jan_dataframe)

# rescale each feature to range [0,1].
scaledData = scalerModel.transform(Jan_dataframe)
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
Jan_scaled= scaledData.rdd.map(lambda data: [data[0],data[2]])
Jan_scaled.toDF(['id', 'features']).show()
Jan_scaledVector= scaledData.rdd.map(lambda data: list(data[2]))

2. Analysis of users'roles on github based on user behaviors
======
This research is aimed at identify different users and their composition. The data is the number of different event that a Github user performed in Jan 2017. The main methods employed is the k-means model. The reason of choosing k-means is that it is a easy understanding, efficient and scalable clustering algorithm. And it has good results as long as the parameters are set correctly. In addition, the K-means||, which is the scalable version of k-means++ is adopted. K-means++ algorithm choose initial values to feed the k-means clustering model and can avoid poor clusterings found by the standard k-means algorithm.

The validation of my approach is determining the number of clusters. Two metrics, WSSSE and Beta-CV are used for the validation of K. These two metrics will be explained in detail in the following part.

##2.1 Calculation of Beta-CV and selection of the number of K

###2.1.1 Intra/Inter cluster CV and their derivation###

**Intra cluster distance: The average intracluster distance for cluster k, deoned as the average distance of all points of cluster k to its centroid**

![alt text](https://github.com/ZXdatascience/MIE-1512/raw/master/Screenshot%20from%202017-04-18%2000-26-40.png "Logo Title Text 1")

*where d(x, Ck) is the distance between point x and the cluster centeroid that point x belongs to. s(k) is the number of points of cluster k*

**Intra Cluster sample mean, sample variance and sample coeffiecient of variance**

![alt text](https://github.com/ZXdatascience/MIE-1512/raw/master/Screenshot%20from%202017-04-18%2000-27-31.png "Logo Title Text 2")

*where k is the number of clusters*

**Inter cluster distance: The average distance between clusters' centers.**

**Inter Cluster sample mean, sample variance and sample coeffiecient of variance**

![alt text](https://github.com/ZXdatascience/MIE-1512/raw/master/Screenshot%20from%202017-04-24%2016-50-11.png "Logo Title Text 2")

![alt text](https://github.com/ZXdatascience/MIE-1512/raw/master/Screenshot%20from%202017-04-24%2016-50-42.png "Logo Title Text 2")

![alt text](https://github.com/ZXdatascience/MIE-1512/raw/master/Screenshot%20from%202017-04-24%2016-50-54.png "Logo Title Text 2")

*where D i,j is the intercluster distance between clusters i and j, for i != j.*

###2.1.2 Beta-cv and how to use it to determine the number of k


A question that comes along with the K-means: How many clusters should we choose? In [2] and [3] the authors suggest that this question can be answered by examining the variation of two metrics: the intracluster distance (average distance between each cluster point and its centroid) and the intercluster distance (average distance between centroids), both characterized by their Coefficient of Variation (CV). The goal is to minimize intracluster CV while maximizing the intercluster CV. **The ratio between the intracluster CV and the intercluster CV**,denoted by β CV , can help us select the value of K. Varying the number of clusters generates different values for β CV . The best indication for K would be when β CV becomes relatively stable [2, 3].

Below are the functions that claculate Inter/Intra cluster CV described above

In [21]:
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from math import sqrt
import numpy as np
def SUM(LIST):
    a= 0
    for i in range(len(LIST)):
        a+= LIST[i]
    return a
def error(point, center):
    return sqrt(SUM([x**2 for x in (np.array(point) - np.array(center))]))
# Distance between a point and a center point
def error2(point, cluster):
    center = cluster.centers[cluster.predict(point)]
    return sqrt(SUM([x**2 for x in (point - center)]))
  
def InterCluster(cluster):
    Dis_list= []
    for i in range(len(cluster.centers)-1):
        for j in range(i+1, len(cluster.centers)):
            Dis_ij= sqrt(SUM([x**2 for x in (cluster.centers[i]- cluster.centers[j])]))
            Dis_list.append(Dis_ij)
    Mean= np.mean(np.array(Dis_list))
    Sd= np.std(np.array(Dis_list), ddof=1)
    cv= Sd/Mean
    return cv
# Coefficient of variance of distance between different cluster centeroid 
# Input: the trained cluster model. Output: InterCluster cv
def filterCluster(Dataframe, k):
    kCluster= [None] *k
    for i in range(k):
        kCluster[i]= Dataframe.where( Dataframe.clusters==i )
    return kCluster
## partition the dataframe into k parts according to the clusters, return a list of k sub-dataframe.
## output: list of dataframes
def IntraCluster(Data, cluster):
    global Jan_DF
    center= cluster.centers
    k= len(cluster.centers)
    predictions= Data.map(lambda r: [r[0]]+ [cluster.predict(list(r[1]))]+ r[1].tolist())
    print Jan_DF.schema.names
    predict_df= predictions.toDF(['actorID', 'clusters']+ Jan_DF.schema.names[1:])
    Clusters= filterCluster(predict_df, k)
    rowsEachCluster= [x.count() for x in Clusters]
    IntraDist= [x.rdd.map(lambda r: error(list(r[2:]), center[r[1]])).reduce(lambda x,y: x+y) for x in Clusters]
    avgIntraDist= [x/y for x, y in zip(IntraDist, rowsEachCluster)]
    Mean= np.mean(np.array(avgIntraDist))
    Sd= np.std(np.array(avgIntraDist), ddof=1)
    cv= Sd/Mean
    return cv
## Coefficient of variance of distance within each cluster
## Input: the raw dataframe: Jan_scaled, trained cluster model
## Output: IntraCluster cv
def Beta_cv(Data, cluster):
    IntraCV= IntraCluster(Data, cluster)
    InterCV= InterCluster(cluster)
    return IntraCV/InterCV
    


##2.2. Train the K-means model and calculate WSSSE and Beta-cv.

###2.2.1 Train the model when k=3,4,5,6,7,8

In this section, the model of Kmeans, which is available and also scalable on spark MLlib is adopted to perform the clustering. The euclidean distance is used to measure the distance between point and point. In addition, k-means||, which is a scalable version of K-means++ is set as the initializationMode.

In [24]:
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from math import sqrt
clusters_k3 = KMeans.train(Jan_scaledVector, 3, maxIterations=25, initializationMode="k-means||" )

clusters_k4 = KMeans.train(Jan_scaledVector, 5, maxIterations=25, initializationMode="k-means||" )

clusters_k5 = KMeans.train(Jan_scaledVector, 5, maxIterations=25, initializationMode="k-means||" )

clusters_k6 = KMeans.train(Jan_scaledVector, 6, maxIterations=25, initializationMode="k-means||" )

clusters_k7 = KMeans.train(Jan_scaledVector, 7, maxIterations=25, initializationMode="k-means||" )

clusters_k8 = KMeans.train(Jan_scaledVector, 8, maxIterations=25, initializationMode="k-means||" )

clusters_k9 = KMeans.train(Jan_scaledVector, 9, maxIterations=25, initializationMode="k-means||" )

###2.2.2 Calculate WSSSE of each number of K

WSSSE is the abbreviation of Within Set Sum of Squared Errors. In another word, it is the sum of the distance of each point with its corresponding center. The lower the value of WSSSE, points are in general closer to their centeroids. Thus, lower WSSSE means better results of K-means.

In [26]:
WSSSE_k3 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k3)).reduce(lambda x,y :x+y)

WSSSE_k4 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k4)).reduce(lambda x,y :x+y)

WSSSE_k5 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k5)).reduce(lambda x,y :x+y)

WSSSE_k6 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k6)).reduce(lambda x,y :x+y)

WSSSE_k7 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k7)).reduce(lambda x,y :x+y)

WSSSE_k8 = Jan_scaled.map(lambda point: error2(list(point[1]), clusters_k8)).reduce(lambda x,y :x+y)

###2.2.3 Calculate the Beta-cv for each k= 3,4,5,6,7,8

In [28]:
beta_cv3= Beta_cv(Jan_scaled, clusters_k3)
print beta_cv3
beta_cv4= Beta_cv(Jan_scaled, clusters_k4)
print beta_cv4
beta_cv5= Beta_cv(Jan_scaled, clusters_k5)
print beta_cv5
beta_cv6= Beta_cv(Jan_scaled, clusters_k6)
print beta_cv6
beta_cv7= Beta_cv(Jan_scaled, clusters_k7)
print beta_cv7
beta_cv8= Beta_cv(Jan_scaled, clusters_k8)
print beta_cv8

##2.3 The selection of K

In [30]:
from pyspark.sql import Row

array1 = [Row(K="3", WSSSE= WSSSE_k3),
         Row(K="4", WSSSE= WSSSE_k4),
         Row(K="5", WSSSE= WSSSE_k5),
         Row(K="6", WSSSE= WSSSE_k6),
         Row(K="7", WSSSE= WSSSE_k7),
         Row(K="8", WSSSE= WSSSE_k8)]
WSSSE_Table = sqlContext.createDataFrame(sc.parallelize(array1))
display(WSSSE_Table)

In [31]:
beta_cv= np.array([beta_cv3, beta_cv4, beta_cv5, beta_cv6, beta_cv7, beta_cv8]).tolist()
array2 = [Row(K="3", Beta_CV=beta_cv[0]),
         Row(K="4", Beta_CV=beta_cv[1]),
         Row(K="5", Beta_CV=beta_cv[2]),
         Row(K="6", Beta_CV=beta_cv[3]),
         Row(K="7", Beta_CV=beta_cv[4]),
         Row(K="8", Beta_CV=beta_cv[5])]
Beta_CV_Table = sqlContext.createDataFrame(sc.parallelize(array2))
display(Beta_CV_Table)

First, we look at the value of WSSSE of each number of K, it can be seen that the WSSSE value continues to decrease as the number of K goes up. However, the decresing rate is getting slower as k increase. It is known that the "perfect" situation is the number of K= the number of points. Therefore, the incresing of k must result in the decreasing of WSSSE. Thus, only from the WSSSE value, it is not enough to determine the appropriate k for clustering.

Then we need to use Beta-cv. As I mentioned, the best indication for K would be when β-CV becomes relatively stable. From above figure, it is clear that the β-CV becomes stable at k=6. Combined with the figure of WSSSE, when k=6, the value of WSSSE is also relatively small. 

**Thus, with the sign of becoming stable at k=6 and relatively small value of WSSSE, *k=6* should be the best number of k for the clustering.**

3. Cluster the users with kmeans(k=6)
==

In [34]:
predictions= Jan_scaled.map(lambda r: [r[0]]+ [clusters_k6.predict(list(r[1]))]+ r[1].tolist())
type_1= ['actorID', 'clusters']+ Jan_DF.schema.names[1:]
predict_df= predictions.toDF(type_1)
## feed the data into the k=6 kmeans model and add the clustering results to the input data.

In [35]:
predict_df.createOrReplaceTempView('githubCluster')

4. Analysis of the users roles and their composition
====

In [37]:
%sql
select clusters, count(*)/(select count(*) from githubCluster)
from githubCluster
group by clusters
order by clusters
/* The percentage of number of users for each cluster*/

In [38]:
%sql
select clusters, avg(PushEvent) push, avg(GollumEvent) Gollum, avg(ReleaseEvent) Release, avg(CommitCommentEvent) CommitComment, avg(CreateEvent) create_, avg(PullRequestReviewCommentEvent) PullRequestComment, avg(IssueCommentEvent) IssueComment, avg(DeleteEvent) Delete, avg(IssuesEvent) Issue, avg(ForkEvent) Fork, avg(PublicEvent) Public, avg(MemberEvent) Member, avg(WatchEvent) Watch, avg(PullRequestEvent) PullRequest
from githubCluster
group by  clusters 
order by  clusters
/* the average attributes for each cluster*/

### From above figure, the users of github are divided into following 4 groups

G0- Inactive users: This group of users are inactive and have very low values in all of the events. This kind of users have formed the major part of the Github users. There are over 82% of users that are inactive. This means that unlike other social network, the majority part of Github users do not frequently use it. A reasonalable answer is that github is such a technical website that most people can't really comfortably make use of the resources of github and quit. 

G1 & G2 & G5- Common users: These three groups share very similar patterns but with different values. Among these three, G1 has the highest values, then G2 and the lowest one is G5. However, if we look at the number of users in these three groups, it's acctually in the reverse order. G5 owns %14 of the users, G2 %3 and G1 only  0.5 %. These three groups form the major part of active users on Github. And with the use frequency increasing, the number of users decreased gradually.

G3- Popular repository administrator: This group is very unique as G3 have high values in push, pullRequest, pullRequestComment, IssueComment, issue and Delete. High value in push means they are very active. Futhermore, high values in pullRequestComment, Issue, IssueComment demonstrate that they receive and reply to many issue and pullrequest. Only those users that have administration to some very popular and well-known repositories would have so many issues and pull requests to deal with. Compared to other groups, G3 also have extremely high value of Delete, which again confirmed their identity as repository administrators.

G4- Enthusiastic contributors: This group have the highest value of push. Nonetheless, they have relatively low value in other events. Thus, they are very active in pushing commits but do not have many interests in communicating with others, post issues and make comments.

Although G3 and G4 are very unique and make a lot of contributions to Github, they are only a very small part of Github users (In total about 0.2%). 99% of the users are inactive users and common users. In summary, Github have a substantial number of contributors (18%) while most of the users are inactive. These active users are the major force of the thriving of Github. Among these active users, two groups: G3 &G4 that only take a very tiny part of all the users, make huge contribution to the website's growth.

In [40]:
%sql
select clusters, avg(GollumEvent) Gollum
from githubCluster
group by  clusters 
order by  clusters
/* the averge gollumEvent value for each cluster*/

In [41]:
%sql
select clusters, avg(MemberEvent) Member
from githubCluster
group by  clusters 
order by  clusters
/* the averge memberEvent value for each cluster*/

The Gollum event is triggered when a Wiki page is created or updated.

The Member event is triggered when a user is added or removed as a collaborator to a repository, or has their permissions changed .

As G3 has the highest value of Gollum and member event, it confirmed that groups are formed from popular repositories administrators. Because only these users will have the right to write a Wiki page and to manage members of the repository.

In [43]:
%sql
select clusters, avg(PublicEvent) Public
from githubCluster
group by  clusters 
order by  clusters
/* the averge publicEvent value for each cluster*/

The publicEvent is Triggered when a private repository is open sourced. Unlike other events, publicEvent is quite evenly distributed among clusters except cluster 0 (inactive group) has very low values. It delivered the message that dispite different users and behaviors, they are all intended to open-source their works. That is a great thing and is the reason why Github developed so rapidly.

5. Conclusions
===
In this report, the data of the numeber of events that performed by the Github users in January 2017 is extracted from Github Archive. The data is then feed into K-means model. Beta-CV and WSSSE are adpted to determine the number of K and then 6 is decided to be the optimal number of K. After clustering the users into 6 groups, the attributes of each cluster and clusters' composition are analyzed.

It is shown that the majority of Github users are inactive users and there are about 18% active users. Among the active users, there is a very tiny part that make great contribution to the repositories' management and growth. Identifying the user behavior has the potential to improve the user experience and accelerate the growth of the community. It also provide information for the customized recommendation system.

References
===
[1] Maia, Marcelo, Jussara Almeida, and Virgílio Almeida. "Identifying user behavior in online social networks." Proceedings of the 1st workshop on Social network systems. ACM, 2008.

[2] D. Menascé and V. Almeida. Scaling for E-Business:
Technologies, Models, Performance, and Capacity
Planning. Upper Saddle River, Prentice Hall, NJ,
2000.

[3] D. Menascé, V. Almeida, R. Fonseca, and M. Mendes.
A Methodology for Workload Characterization of
E-Commerce Sites. In Proc. ACM Conference on
Electronic Commerce (EC), pages 119–128, 1999.